In [1]:
import os
import warnings
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (10, 6)
sns.set(style='whitegrid')

In [8]:
# -----------------------------
# Configuration
# -----------------------------
DATA_PATH = DATA_PATH = r"C:\Users\Lenovo\Downloads\housing_data (1).csv"
OUTPUT_DIR = 'eda_outputs'
CURRENT_YEAR = datetime.now().year
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [9]:
# -----------------------------
# Helper functions
# -----------------------------

def load_data(path: str) -> pd.DataFrame:
    """Load CSV (or Excel) into a DataFrame."""
    if path.endswith('.csv'):
        return pd.read_csv(path)
    elif path.endswith(('.xls', '.xlsx')):
        return pd.read_excel(path)
    else:
        raise ValueError('Unsupported file format. Use CSV or Excel.')


def save_fig(fig, name):
    path = os.path.join(OUTPUT_DIR, name)
    fig.savefig(path, bbox_inches='tight', dpi=150)
    plt.close(fig)


In [11]:
# -----------------------------
# 1. Load the data
# -----------------------------
print('Loading data from',DATA_PATH )
try:
    df = load_data(DATA_PATH)
except Exception as e:
    raise SystemExit(f"Failed to load data: {e}")

print('Initial rows:', df.shape[0], 'columns:', df.shape[1])

# Quick peek
print('\nColumns:')
print(df.columns.tolist())
print('\nData types:')
print(df.dtypes)

Loading data from C:\Users\Lenovo\Downloads\housing_data (1).csv
Initial rows: 1460 columns: 81

Columns:
['Unnamed: 0', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageC

In [12]:
# -----------------------------
# 2. Data cleaning
# -----------------------------
# 2.1 Standardize column names
orig_cols = df.columns.tolist()
cols = [c.strip().lower().replace(' ', '_').replace('(', '').replace(')', '') for c in orig_cols]
rename_map = dict(zip(orig_cols, cols))
df.rename(columns=rename_map, inplace=True)
print('\nRenamed columns to standardized format.')

# 2.2 Remove exact duplicates
before_dup = df.shape[0]
df.drop_duplicates(inplace=True)
after_dup = df.shape[0]
print(f'Dropped {before_dup - after_dup} duplicate rows')

# 2.3 Identify missingness
missing = df.isna().sum().sort_values(ascending=False)
print('\nTop missing values:')
print(missing[missing > 0].head(20))

# Strategy (generic):
# - If a critical numeric column (price, area) has missing -> drop or impute with median
# - If categorical -> fill with 'Unknown' or mode

# Guess common column names; adapt if dataset differs
possible_price_cols = [c for c in df.columns if 'price' in c]
possible_area_cols = [c for c in df.columns if any(x in c for x in ['area', 'sqft', 'size', 'built_up'])]
print('\nDetected price-like columns:', possible_price_cols)
print('Detected area-like columns:', possible_area_cols)

# We'll pick first matches if available
price_col = possible_price_cols[0] if possible_price_cols else None
area_col = possible_area_cols[0] if possible_area_cols else None

if price_col is None or area_col is None:
    print('\nWarning: Could not auto-detect price or area columns. Please update the script with the correct column names.')

# Convert price & area to numeric
if price_col:
    df[price_col] = pd.to_numeric(df[price_col], errors='coerce')
if area_col:
    df[area_col] = pd.to_numeric(df[area_col], errors='coerce')

# Drop rows missing both price and area - cannot usefully analyze
if price_col and area_col:
    before = df.shape[0]
    df = df.dropna(subset=[price_col, area_col], how='any')
    after = df.shape[0]
    print(f'Dropped {before-after} rows missing price or area')

# Impute numeric columns with median
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_imputer = SimpleImputer(strategy='median')
if len(numeric_cols) > 0:
    df[numeric_cols] = pd.DataFrame(num_imputer.fit_transform(df[numeric_cols]), columns=numeric_cols)

# Fill categorical NaNs with 'Unknown'
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
for c in cat_cols:
    df[c] = df[c].fillna('Unknown')

print('\nAfter cleaning, dataset shape:', df.shape)



Renamed columns to standardized format.
Dropped 0 duplicate rows

Top missing values:
alley          1369
masvnrtype      872
garageyrblt      81
electrical        1
dtype: int64

Detected price-like columns: ['saleprice']
Detected area-like columns: ['lotarea', 'masvnrarea', 'grlivarea', 'garagearea', 'poolarea']
Dropped 0 rows missing price or area

After cleaning, dataset shape: (1460, 81)


In [14]:
# -----------------------------
# 3. Univariate analysis
# -----------------------------
# Price distribution
if price_col:
    fig = plt.figure()
    sns.histplot(df[price_col], kde=True)
    plt.title('Distribution of Price')
    save_fig(fig, 'price_distribution.png')

    fig = plt.figure()
    sns.boxplot(x=df[price_col])
    plt.title('Price Boxplot (to show outliers)')
    save_fig(fig, 'price_boxplot.png')

# Area distribution
if area_col:
    fig = plt.figure()
    sns.histplot(df[area_col], kde=True)
    plt.title('Distribution of Area')
    save_fig(fig, 'area_distribution.png')

# Categorical value counts (top categories)
for c in cat_cols[:6]:
    fig = plt.figure()
    sns.countplot(y=c, data=df, order=df[c].value_counts().iloc[:15].index)
    plt.title(f'Top categories for {c}')
    save_fig(fig, f'count_{c}.png')


In [16]:
# -----------------------------
# 4. Multivariate analysis
# -----------------------------
# Correlation heatmap for numeric features
if len(numeric_cols) > 1:
    corr = df[numeric_cols].corr()
    fig = plt.figure(figsize=(12, 10))
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
    plt.title('Correlation Matrix')
    save_fig(fig, 'correlation_matrix.png')

# Scatter plot price vs area
if price_col and area_col:
    fig = plt.figure()
    sns.scatterplot(x=df[area_col], y=df[price_col])
    plt.title('Price vs Area')
    save_fig(fig, 'price_vs_area.png')

# Price by bedrooms/bathrooms if columns exist
bed_cols = [c for c in df.columns if 'bed' in c]
bath_cols = [c for c in df.columns if 'bath' in c]

if bed_cols:
    b = bed_cols[0]
    fig = plt.figure()
    sns.boxplot(x=b, y=price_col, data=df)
    plt.title('Price by Number of Bedrooms')
    save_fig(fig, 'price_by_bedrooms.png')
    
if bath_cols:
    b = bath_cols[0]
    fig = plt.figure()
    sns.boxplot(x=b, y=price_col, data=df)
    plt.title('Price by Number of Bathrooms')
    save_fig(fig, 'price_by_bathrooms.png')


In [18]:
# -----------------------------
# 5. Feature engineering
# -----------------------------
# Price per sqft
if price_col and area_col:
    df['price_per_sqft'] = df[price_col] / df[area_col].replace({0: np.nan})
    df['price_per_sqft'].fillna(df['price_per_sqft'].median(), inplace=True)

# Age of property from year_built or built_year columns
year_cols = [c for c in df.columns if any(x in c for x in ['year', 'built', 'construction'])]
if year_cols:
    ycol = year_cols[0]
    # ensure numeric
    df[ycol] = pd.to_numeric(df[ycol], errors='coerce')
    df['property_age'] = CURRENT_YEAR - df[ycol]
    df['property_age'] = df['property_age'].clip(lower=0)
else:
    df['property_age'] = np.nan

# Binary flags for common amenities
amenities = ['pool', 'garage', 'parking', 'balcony', 'garden', 'gym']
for a in amenities:
    cols_with_a = [c for c in df.columns if a in c]
    if cols_with_a:
        c0 = cols_with_a[0]
        df[f'has_{a}'] = df[c0].astype(str).str.lower().apply(lambda x: 1 if x not in ['none', 'no', '0', 'unknown', 'nan'] and x.strip() else 0)
    else:
        df[f'has_{a}'] = 0

# Save engineered feature snapshot
feats_to_save = ['price_per_sqft', 'property_age'] + [f'has_{a}' for a in amenities if f'has_{a}' in df.columns]
feats_existing = [f for f in feats_to_save if f in df.columns]
df[feats_existing].to_csv(os.path.join(OUTPUT_DIR, 'engineered_features.csv'), index=False)
print('\nSaved engineered features:', feats_existing)


Saved engineered features: ['price_per_sqft', 'property_age', 'has_pool', 'has_garage', 'has_parking', 'has_balcony', 'has_garden', 'has_gym']


In [19]:
# -----------------------------
# 6. Clustering on numeric features (KMeans)
# -----------------------------
cluster_features = [price_col, area_col, 'price_per_sqft'] if price_col and area_col else numeric_cols[:3]
cluster_features = [c for c in cluster_features if c in df.columns]
cluster_df = df[cluster_features].copy().dropna()

if len(cluster_features) >= 2 and cluster_df.shape[0] >= 10:
    scaler = StandardScaler()
    X = scaler.fit_transform(cluster_df)
    kmeans = KMeans(n_clusters=3, random_state=42)
    labels = kmeans.fit_predict(X)
    cluster_df['cluster'] = labels
    # Merge back a small sample label column
    df.loc[cluster_df.index, 'cluster_label'] = labels
    fig = plt.figure()
    sns.scatterplot(x=cluster_df[cluster_features[1]], y=cluster_df[cluster_features[0]], hue=cluster_df['cluster'], palette='deep')
    plt.title('KMeans clusters')
    save_fig(fig, 'kmeans_clusters.png')
    print('KMeans clustering done with features:', cluster_features)
else:
    print('Not enough numeric features or rows for clustering. Skipping KMeans.')


KMeans clustering done with features: ['saleprice', 'lotarea', 'price_per_sqft']


In [20]:
# -----------------------------
# 7. Simple predictive model: Linear Regression
# -----------------------------
# Use a small subset of features
model_features = []
if area_col:
    model_features.append(area_col)
if 'price_per_sqft' in df.columns:
    model_features.append('price_per_sqft')
if 'property_age' in df.columns:
    model_features.append('property_age')
# add numeric amenity flags
model_features += [f'has_{a}' for a in amenities if f'has_{a}' in df.columns]
model_features = [m for m in model_features if m in df.columns]

if price_col and len(model_features) >= 1:
    X = df[model_features]
    y = df[price_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    preds = lr.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, preds)
    print(f'Linear Regression results: RMSE={rmse:.2f}, R2={r2:.3f}')
    # Coefficients
    coef_df = pd.DataFrame({'feature': model_features, 'coefficient': lr.coef_})
    coef_df.to_csv(os.path.join(OUTPUT_DIR, 'lr_coefficients.csv'), index=False)
    print('Saved linear regression coefficients to lr_coefficients.csv')
else:
    print('Insufficient data to train linear regression. Need price column and at least one feature.')


Linear Regression results: RMSE=68244.55, R2=0.393
Saved linear regression coefficients to lr_coefficients.csv


In [22]:
# -----------------------------
# 8. Market trends (time-series)
# -----------------------------
# If there's a transaction_date or year column, aggregate by year/month
date_cols = [c for c in df.columns if 'date' in c or 'transaction' in c or 'sold' in c]
if date_cols:
    dcol = date_cols[0]
    try:
        df[dcol] = pd.to_datetime(df[dcol], errors='coerce')
        df['year_month'] = df[dcol].dt.to_period('M')
        ts = df.groupby('year_month')[price_col].median().reset_index()
        ts['year_month'] = ts['year_month'].dt.to_timestamp()
        fig = plt.figure()
        sns.lineplot(x='year_month', y=price_col, data=ts)
        plt.title('Median Price over Time')
        plt.xticks(rotation=45)
        save_fig(fig, 'price_over_time.png')
    except Exception as e:
        print('Failed to parse date column for time-series:', e)
else:
    print('No obvious date column found for time-series analysis.')


In [23]:
# -----------------------------
# 9. Amenity impact analysis example
# -----------------------------
for a in amenities:
    col = f'has_{a}'
    if col in df.columns:
        agg = df.groupby(col)[price_col].agg(['count', 'median', 'mean']).reset_index()
        print(f"\nAmenity impact for {a}:")
        print(agg)
        agg.to_csv(os.path.join(OUTPUT_DIR, f'amenity_impact_{a}.csv'), index=False)



Amenity impact for pool:
   has_pool  count    median          mean
0         1   1460  163000.0  180921.19589

Amenity impact for garage:
   has_garage  count    median           mean
0           0     81  100000.0  103317.283951
1           1   1379  167500.0  185479.511240

Amenity impact for parking:
   has_parking  count    median          mean
0            0   1460  163000.0  180921.19589

Amenity impact for balcony:
   has_balcony  count    median          mean
0            0   1460  163000.0  180921.19589

Amenity impact for garden:
   has_garden  count    median          mean
0           0   1460  163000.0  180921.19589

Amenity impact for gym:
   has_gym  count    median          mean
0        0   1460  163000.0  180921.19589


In [24]:
# -----------------------------
# 10. Save cleaned dataset & summary
# -----------------------------
clean_path = os.path.join(OUTPUT_DIR, 'cleaned_housing_data.csv')
df.to_csv(clean_path, index=False)
print('\nSaved cleaned dataset to', clean_path)

summary = {
    'initial_rows': before_dup,
    'final_rows': df.shape[0],
    'num_numeric_cols': len(numeric_cols),
    'num_categorical_cols': len(cat_cols)
}

with open(os.path.join(OUTPUT_DIR, 'summary.txt'), 'w') as f:
    for k, v in summary.items():
        f.write(f"{k}: {v}\n")

print('\nEDA complete. Visualizations and artifacts saved in the directory:', OUTPUT_DIR)
print('Check the files (png, csv, summary.txt) for the outputs.')


Saved cleaned dataset to eda_outputs\cleaned_housing_data.csv

EDA complete. Visualizations and artifacts saved in the directory: eda_outputs
Check the files (png, csv, summary.txt) for the outputs.
